<a href="https://colab.research.google.com/github/ewotawa/secure_private_ai/blob/master/Section_2_Federated_Learning_Final_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Federated Learning Final Project

## Overview
* See  <a href="https://classroom.udacity.com/nanodegrees/nd185/parts/3fe1bb10-68d7-4d84-9c99-9539dedffad5/modules/28d685f0-0cb1-4f94-a8ea-2e16614ab421/lessons/c8fe481d-81ea-41be-8206-06d2deeb8575/concepts/a5fb4b4c-e38a-48de-b2a7-4e853c62acbe">video</a> for additional details. 
* Do Federated Learning where the central server is not trusted with the raw gradients.  
* In the final project notebook, you'll receive a dataset.  
* Train on the dataset using Federated Learning.  
* The gradients should not come up to the server in raw form.  
* Instead, use the new .move() command to move all of the gradients to one of the workers, sum them up there, and then bring that batch up to the central server and then bring that batch up 
* Idea: the central server never actually sees the raw gradient for any person.  
* We'll look at secure aggregation in course 3.  
* For now, do a larger-scale Federated Learning case where you handle the gradients in a special way.

## Approach
* Reviewing methods of classmates for Federated Learning. 

## References
*  <a href = "https://github.com/edgarinvillegas/private-ai/blob/master/Section%203%20-%20Final%20project.ipynb/">GitHub Notebook</a>
* <a href = "https://github.com/OpenMined/PySyft/blob/dev/examples/tutorials/Part%2010%20-%20Federated%20Learning%20with%20Secure%20Aggregation.ipynb">Part 10: Federated Learning with Encrypted Gradient Aggregation</a>


### Install libraries and dependencies

In [1]:
# PySyft

!pip install syft

import syft as sy

# PyTorch

!pip install torch
!pip install torchvision

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import torchvision
from torchvision import datasets, transforms

# Numpy

import numpy as np

# time

import time

W0723 00:47:31.931035 140324268464000 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0723 00:47:31.951922 140324268464000 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



### Normal Federated Learning

In [0]:
# Establish a parser to contain parameters for training

class Parser: 
  def __init__(self):
    self.epochs = 10
    self.lr = 0.001
    self.test_batch_size = 1
    self.batch_size = 1
    self.log_interval = 10
    self.seed = 1
    
args = Parser()

torch.manual_seed(args.seed)

kwargs = {}

In [0]:
# A Toy Dataset

data = torch.randn(50, 2, requires_grad=True) 
target = torch.randn(50, 1, requires_grad=True) 

data_test = torch.randn(50, 2, requires_grad=True) 
target_test = torch.randn(50, 1, requires_grad=True) 

In [0]:
train = TensorDataset(data, target)
test = TensorDataset(data_test, target_test)

trainloader = DataLoader(train, batch_size=args.batch_size, shuffle=True, **kwargs)
testloader = DataLoader(test, batch_size=args.test_batch_size, shuffle=True, **kwargs)

In [5]:
# A Toy Model

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(2,1)

  def forward(self, x):
    x = self.fc1(x)
    return x

# model = nn.Linear(2,1)

model = Net() 

print(model)

Net(
  (fc1): Linear(in_features=2, out_features=1, bias=True)
)


In [6]:
# Optimizer
opt = optim.SGD(params=model.parameters(), lr=args.lr)
print(opt)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0
    nesterov: False
    weight_decay: 0
)


Set up hook, virtual workers, and virtual aggregator

In [0]:
hook = sy.TorchHook(torch)
vw00 = sy.VirtualWorker(hook, id="vw00")
vw01 = sy.VirtualWorker(hook, id="vw01")

aggr = sy.VirtualWorker(hook, id="aggr")

compute_nodes = [vw00, vw01]

In [8]:
vw00.clear_objects()
vw01.clear_objects()
aggr.clear_objects()

<VirtualWorker id:aggr #objects:0>

In [9]:
# Send data to the workers
train_dist_dataset = []

for batch_idx, (data, target) in enumerate(trainloader):
  data = data.send(compute_nodes[batch_idx % len(compute_nodes)])
  target = target.send(compute_nodes[batch_idx % len(compute_nodes)])
  train_dist_dataset.append((data, target))
  
print(train_dist_dataset)

[((Wrapper)>[PointerTensor | me:87852161962 -> vw00:1867014711], (Wrapper)>[PointerTensor | me:54068292559 -> vw00:95763329025]), ((Wrapper)>[PointerTensor | me:21917541789 -> vw01:61682282552], (Wrapper)>[PointerTensor | me:31214785569 -> vw01:23371270086]), ((Wrapper)>[PointerTensor | me:70783505929 -> vw00:15790465499], (Wrapper)>[PointerTensor | me:63261927165 -> vw00:63101312369]), ((Wrapper)>[PointerTensor | me:67037946874 -> vw01:7204808435], (Wrapper)>[PointerTensor | me:78087928504 -> vw01:9111702806]), ((Wrapper)>[PointerTensor | me:37855987236 -> vw00:65061298900], (Wrapper)>[PointerTensor | me:27448136907 -> vw00:10783643674]), ((Wrapper)>[PointerTensor | me:55169964551 -> vw01:60578336403], (Wrapper)>[PointerTensor | me:91253173734 -> vw01:43847923775]), ((Wrapper)>[PointerTensor | me:88007669934 -> vw00:43672854421], (Wrapper)>[PointerTensor | me:29930836746 -> vw00:80572784492]), ((Wrapper)>[PointerTensor | me:24262887699 -> vw01:46582995190], (Wrapper)>[PointerTensor | 

In [0]:
# define loss function

def loss_func(pred, target):
  loss = ((pred - target)**2).sum()
  return loss

In [0]:
# training function

def train(epoch):
  model.train()
  
  for batch_idx, (data, target) in enumerate(train_dist_dataset):
    # determine the active worker
    worker = data.location
    # send the model to the active worker
    model.send(worker)
    
    # do normal training
    opt.zero_grad()
    pred = model(data)
    loss = loss_func(pred, target)
    loss.backward()
    opt.step()
    model.get()

    if batch_idx % args.log_interval == 0:
      loss = loss.get()
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tloss: {:.6f}'.format(
        epoch, batch_idx, len(trainloader),
          100. * batch_idx / len(trainloader), loss.item()))

In [0]:
# testing function

def test():
  model.eval()
  test_loss = 0
  for data, target in testloader:
    pred = model(data)
    test_loss += loss_func(pred, target).item() # sum up batch loss
    
  test_loss /= len(testloader.dataset)
  print('\nTest Set: Average Loss: {:.4f}\n'.format(test_loss))

In [13]:
# train the model

t = time.time()

for epoch in range(1, args.epochs + 1):
  train(epoch)
  
total_time = time.time() - t
print('total', round(total_time, 2), 'sec')

Train Epoch: 1 [0/50 (0%)]	loss: 2.548004
Train Epoch: 1 [10/50 (20%)]	loss: 1.465525
Train Epoch: 1 [20/50 (40%)]	loss: 1.308707
Train Epoch: 1 [30/50 (60%)]	loss: 0.103834
Train Epoch: 1 [40/50 (80%)]	loss: 0.000071
Train Epoch: 2 [0/50 (0%)]	loss: 2.288096
Train Epoch: 2 [10/50 (20%)]	loss: 1.318819
Train Epoch: 2 [20/50 (40%)]	loss: 1.319915
Train Epoch: 2 [30/50 (60%)]	loss: 0.120199
Train Epoch: 2 [40/50 (80%)]	loss: 0.000036
Train Epoch: 3 [0/50 (0%)]	loss: 2.069994
Train Epoch: 3 [10/50 (20%)]	loss: 1.193838
Train Epoch: 3 [20/50 (40%)]	loss: 1.322092
Train Epoch: 3 [30/50 (60%)]	loss: 0.137194
Train Epoch: 3 [40/50 (80%)]	loss: 0.000446
Train Epoch: 4 [0/50 (0%)]	loss: 1.886120
Train Epoch: 4 [10/50 (20%)]	loss: 1.086949
Train Epoch: 4 [20/50 (40%)]	loss: 1.317107
Train Epoch: 4 [30/50 (60%)]	loss: 0.154615
Train Epoch: 4 [40/50 (80%)]	loss: 0.001332
Train Epoch: 5 [0/50 (0%)]	loss: 1.730398
Train Epoch: 5 [10/50 (20%)]	loss: 0.995188
Train Epoch: 5 [20/50 (40%)]	loss: 1.30656

In [14]:
# calculating performance
test()


Test Set: Average Loss: 0.8006



### Add Encrypted Aggregation

In [15]:
# Send data to the workers
remote_dataset = (list(), list())

train_dist_dataset = []

for batch_idx, (data, target) in enumerate(trainloader):
  data = data.send(compute_nodes[batch_idx % len(compute_nodes)])
  target = target.send(compute_nodes[batch_idx % len(compute_nodes)])
  remote_dataset[batch_idx % len(compute_nodes)].append((data, target))
  

print(remote_dataset)

print(train_dist_dataset)

([((Wrapper)>[PointerTensor | me:65002615150 -> vw00:59849677808], (Wrapper)>[PointerTensor | me:26908933916 -> vw00:73078401570]), ((Wrapper)>[PointerTensor | me:54227963716 -> vw00:80580904657], (Wrapper)>[PointerTensor | me:79353357986 -> vw00:12048616746]), ((Wrapper)>[PointerTensor | me:92946594792 -> vw00:70564547381], (Wrapper)>[PointerTensor | me:58280343932 -> vw00:29025160202]), ((Wrapper)>[PointerTensor | me:64868705503 -> vw00:13671108078], (Wrapper)>[PointerTensor | me:83637100468 -> vw00:42348767776]), ((Wrapper)>[PointerTensor | me:84837849091 -> vw00:84962101360], (Wrapper)>[PointerTensor | me:47451228808 -> vw00:98724905608]), ((Wrapper)>[PointerTensor | me:94241518471 -> vw00:34811968038], (Wrapper)>[PointerTensor | me:3102425379 -> vw00:34548988521]), ((Wrapper)>[PointerTensor | me:92625188903 -> vw00:67876750185], (Wrapper)>[PointerTensor | me:56527732384 -> vw00:29039250189]), ((Wrapper)>[PointerTensor | me:90172575284 -> vw00:64774865029], (Wrapper)>[PointerTensor

In [0]:
# need to be able to send distinct model to each worker to train.

def update(data, target, model, optimizer):
  model.send(data.location)
  optimizer.zero_grad()
  pred = model(data)
  loss = loss_func(pred, target)
  loss.backward()
  optimizer.step()
  return model

In [0]:
# define worker-based models and optimizer

vw00_model = Net()
vw01_model = Net()

vw00_optimizer = optim.SGD(vw00_model.parameters(), lr=args.lr)
vw01_optimizer = optim.SGD(vw01_model.parameters(), lr=args.lr)

models = [vw00_model, vw01_model]
params = [list(vw00_model.parameters()), list(vw01_model.parameters())]
optimizers = [vw00_optimizer, vw01_optimizer]

#### Train

In [0]:
# select batch on which to train
data_index = 0

# update remote models
# here, iterate once per model

for remote_index in range(len(compute_nodes)):
  data, target = remote_dataset[remote_index][data_index]
  models[remote_index] = update(data, target, models[remote_index], optimizers[remote_index])

#### Encrypted Aggregation

In [0]:
# list to store encrypted model average
new_params = list()

In [20]:
# visualize the parameters
for i in range(len(params[0])):
  
  # for each worker
  spdz2_params = list()
  for remote_index in range(len(compute_nodes)):
    
    lcop = params[remote_index][i].copy().get() 
    
    print("\nremote index: ", remote_index, "\n")
    print(lcop)
    print(lcop.type())
    
    lcop_th = lcop * 10000
    print(lcop_th)
    
    lcop_fx = lcop_th.type(torch.LongTensor)
    print(lcop_fx)
    print(lcop_fx.type())
    
    lcop_th_fl = lcop_fx.type(torch.FloatTensor)
    lcop_fl = lcop_th_fl / 10000
    print(lcop_fl)
    print(lcop_fl.type())


remote index:  0 

tensor([[-0.6983, -0.5240]], requires_grad=True)
torch.FloatTensor
tensor([[-6983.4028, -5239.7515]], grad_fn=<MulBackward0>)
tensor([[-6983, -5239]])
torch.LongTensor
tensor([[-0.6983, -0.5239]])
torch.FloatTensor

remote index:  1 

tensor([[-0.2316, -0.2829]], requires_grad=True)
torch.FloatTensor
tensor([[-2316.2527, -2829.3748]], grad_fn=<MulBackward0>)
tensor([[-2316, -2829]])
torch.LongTensor
tensor([[-0.2316, -0.2829]])
torch.FloatTensor

remote index:  0 

tensor([0.3239], requires_grad=True)
torch.FloatTensor
tensor([3239.4980], grad_fn=<MulBackward0>)
tensor([3239])
torch.LongTensor
tensor([0.3239])
torch.FloatTensor

remote index:  1 

tensor([-0.5834], requires_grad=True)
torch.FloatTensor
tensor([-5834.4795], grad_fn=<MulBackward0>)
tensor([-5834])
torch.LongTensor
tensor([-0.5834])
torch.FloatTensor


In [21]:
# iterate through each parameter
for i in range(len(params[0])):
  
  # for each worker
  spdz_params_th = list()
  for remote_index in range(len(compute_nodes)):
    
    # copy of parameter (cop): copy same parameter from each worker's model, floating type tensor
    cop = params[remote_index][i].copy()
    
    # copy of parameter, thousands (cop_th): copy of parameter scaled up so that precision is not lost
    # fixed precision parameter, thousands (fpp_th): fixed precision version of parameter scaled up so that precision is not lost
    fpp_th = cop * 10000
    
    # encrypt on the remote machine. Note: fixed_precision_param is already a pointer. 
    # calling share encrypts data to which pointer is pointing. Returns a pointer to the MPC secret shared object. Need to fetch object.
    encrypted_param_th = fpp_th.share(vw00, vw01, crypto_provider=aggr)
    
    # fetch the pointer to the MPC shared value
    param_th = encrypted_param_th.get()
    
    # save parameter so can average with same parameter from other workers.
    spdz_params_th.append(param_th)
    
  # average param from multiple workers, fetch back to local machine. Decrypt and decode from fixed precision back to float. 
  new_param_th = (spdz_params_th[0] + spdz_params_th[1]).get() / 2 #.float_precision() / 2 
  new_param_float_th = new_param_th.type(torch.FloatTensor) 
  new_param_float = new_param_float_th / 10000
  
  # save new averaged parameter
  new_params.append(new_param_float)

print(new_params)



[tensor([[-0.4649, -0.4034]]), tensor([-0.1297])]


#### Cleanup

In [0]:
with torch.no_grad():
  for model in params:
    for param in model:
      param *= 0
  
  for model in models:
    model.get()
    
  for remote_index in range(len(compute_nodes)):
    for param_index in range(len(params[remote_index])):
      params[remote_index][param_index].set_(new_params[param_index])

### Bring components together

In [131]:
vw00.clear_objects()
vw01.clear_objects()
aggr.clear_objects()

<VirtualWorker id:aggr #objects:0>

In [132]:
# Send data to the workers
remote_dataset = (list(), list())

train_dist_dataset = []

for batch_idx, (data, target) in enumerate(trainloader):
  data = data.send(compute_nodes[batch_idx % len(compute_nodes)])
  target = target.send(compute_nodes[batch_idx % len(compute_nodes)])
  remote_dataset[batch_idx % len(compute_nodes)].append((data, target))
  

print(remote_dataset)

print(train_dist_dataset)

([((Wrapper)>[PointerTensor | me:13055735074 -> vw00:7549626001], (Wrapper)>[PointerTensor | me:47763213477 -> vw00:93877278802]), ((Wrapper)>[PointerTensor | me:18084543708 -> vw00:97643729603], (Wrapper)>[PointerTensor | me:82307066861 -> vw00:68783321413]), ((Wrapper)>[PointerTensor | me:89837241357 -> vw00:54674009703], (Wrapper)>[PointerTensor | me:48466508546 -> vw00:15375753952]), ((Wrapper)>[PointerTensor | me:98107722951 -> vw00:28978839437], (Wrapper)>[PointerTensor | me:32732767546 -> vw00:83574048018]), ((Wrapper)>[PointerTensor | me:77890224790 -> vw00:30417816088], (Wrapper)>[PointerTensor | me:27102850341 -> vw00:90764472644]), ((Wrapper)>[PointerTensor | me:45813438738 -> vw00:30730983947], (Wrapper)>[PointerTensor | me:9445357400 -> vw00:78181753320]), ((Wrapper)>[PointerTensor | me:54209475732 -> vw00:55531862620], (Wrapper)>[PointerTensor | me:15217953896 -> vw00:17444441723]), ((Wrapper)>[PointerTensor | me:15178413090 -> vw00:88607288440], (Wrapper)>[PointerTensor 

In [133]:
print('length of remote dataset:\t', len(remote_dataset))
print('length of one item in remote dataset:\t', len(remote_dataset[0]))

length of remote dataset:	 2
length of one item in remote dataset:	 25


In [0]:
# define worker-based models and optimizer

vw00_model = Net().send(vw00)
vw01_model = Net().send(vw01)

vw00_optimizer = optim.SGD(vw00_model.parameters(), lr=args.lr)
vw01_optimizer = optim.SGD(vw01_model.parameters(), lr=args.lr)

models = [vw00_model, vw01_model]
params = [list(vw00_model.parameters()), list(vw01_model.parameters())]
optimizers = [vw00_optimizer, vw01_optimizer]

In [0]:
# need to be able to send distinct model to each worker to train.

def update2(data, target, model, optimizer):
  # model.send(data.location)
  optimizer.zero_grad()
  pred = model(data)
  loss = loss_func(pred, target)
  loss.backward()
  optimizer.step()
  return model

In [136]:
# figure out the guts of the training mechanism before putting into function.

for data_index in range(len(remote_dataset[0])):
  
  # update remote models
  
  for remote_index in range(len(compute_nodes)):
    print('data index: \t', data_index, '\tremote index:\t', remote_index)
    data, target = remote_dataset[remote_index][data_index]
    model = models[remote_index]
    opt = optimizers[remote_index]
    
    models[remote_index] = update2(data, target, model, opt)
    
  # encrypted aggregation
  new_params = list() # list to store encrypted model average
  
  for i in range(len(params[0])): # iterate through each parameter

    # for each worker
    spdz_params_th = list()
    for remote_index in range(len(compute_nodes)):

      # copy of parameter (cop): copy same parameter from each worker's model, floating type tensor
      cop = params[remote_index][i].copy()

      # copy of parameter, thousands (cop_th): copy of parameter scaled up so that precision is not lost
      # fixed precision parameter, thousands (fpp_th): fixed precision version of parameter scaled up so that precision is not lost
      fpp_th = cop * 10000

      # encrypt on the remote machine. Note: fixed_precision_param is already a pointer. 
      # calling share encrypts data to which pointer is pointing. Returns a pointer to the MPC secret shared object. Need to fetch object.
      encrypted_param_th = fpp_th.share(vw00, vw01, crypto_provider=aggr)

      # fetch the pointer to the MPC shared value
      param_th = encrypted_param_th.get()

      # save parameter so can average with same parameter from other workers.
      spdz_params_th.append(param_th)

    # average param from multiple workers, fetch back to local machine. Decrypt and decode from fixed precision back to float. 
    new_param_th = (spdz_params_th[0] + spdz_params_th[1]).get() / 2 #.float_precision() / 2 
    new_param_float_th = new_param_th.type(torch.FloatTensor) 
    new_param_float = new_param_float_th / 10000

    # save new averaged parameter
    new_params.append(new_param_float)

  print(new_params)  
  
  with torch.no_grad():
    for model in params:
      for param in model:
        param *= 0
      
    for remote_index in range(len(compute_nodes)):
      for param_index in range(len(params[remote_index])):
        params[remote_index][param_index].set_(new_params[param_index])
  
  

data index: 	 0 	remote index:	 0
data index: 	 0 	remote index:	 1
[tensor([[ 0.1853, -0.4887]]), tensor([0.1713])]
data index: 	 1 	remote index:	 0
data index: 	 1 	remote index:	 1
[tensor([[0.0010, 0.0000]]), tensor([-0.0007])]
data index: 	 2 	remote index:	 0
data index: 	 2 	remote index:	 1
[tensor([[0.0007, 0.0000]]), tensor([-0.0013])]
data index: 	 3 	remote index:	 0
data index: 	 3 	remote index:	 1
[tensor([[9.0000e-04, 1.0000e-04]]), tensor([0.0003])]
data index: 	 4 	remote index:	 0
data index: 	 4 	remote index:	 1
[tensor([[ 0.0004, -0.0010]]), tensor([-0.0017])]
data index: 	 5 	remote index:	 0
data index: 	 5 	remote index:	 1
[tensor([[0.0000, 0.0003]]), tensor([-0.0007])]
data index: 	 6 	remote index:	 0
data index: 	 6 	remote index:	 1
[tensor([[0.0003, 0.0010]]), tensor([-0.0012])]
data index: 	 7 	remote index:	 0
data index: 	 7 	remote index:	 1
[tensor([[-8.0000e-04, -1.0000e-04]]), tensor([-1.0000e-04])]
data index: 	 8 	remote index:	 0
data index: 	 

In [0]:
for model in models:
    model.get()

In [143]:
print(models)

[Net(
  (fc1): Linear(in_features=2, out_features=1, bias=True)
), Net(
  (fc1): Linear(in_features=2, out_features=1, bias=True)
)]


#### Function format

In [144]:
vw00.clear_objects()
vw01.clear_objects()
aggr.clear_objects()

<VirtualWorker id:aggr #objects:0>

In [145]:
# Send data to the workers
remote_dataset = (list(), list())

train_dist_dataset = []

for batch_idx, (data, target) in enumerate(trainloader):
  data = data.send(compute_nodes[batch_idx % len(compute_nodes)])
  target = target.send(compute_nodes[batch_idx % len(compute_nodes)])
  remote_dataset[batch_idx % len(compute_nodes)].append((data, target))
  

print(remote_dataset)

print(train_dist_dataset)

([((Wrapper)>[PointerTensor | me:46350504682 -> vw00:11937363747], (Wrapper)>[PointerTensor | me:74542143739 -> vw00:42603377771]), ((Wrapper)>[PointerTensor | me:71735292092 -> vw00:37055709704], (Wrapper)>[PointerTensor | me:97907306886 -> vw00:19249559529]), ((Wrapper)>[PointerTensor | me:96778984353 -> vw00:76485081131], (Wrapper)>[PointerTensor | me:33817271037 -> vw00:75645769899]), ((Wrapper)>[PointerTensor | me:12233494533 -> vw00:96159922248], (Wrapper)>[PointerTensor | me:902334186 -> vw00:76843649898]), ((Wrapper)>[PointerTensor | me:38691482636 -> vw00:46040608577], (Wrapper)>[PointerTensor | me:87532221208 -> vw00:88556117937]), ((Wrapper)>[PointerTensor | me:30857026478 -> vw00:12688120964], (Wrapper)>[PointerTensor | me:98441724988 -> vw00:1300317819]), ((Wrapper)>[PointerTensor | me:86816668536 -> vw00:40076269591], (Wrapper)>[PointerTensor | me:72725928197 -> vw00:25737436281]), ((Wrapper)>[PointerTensor | me:91851539260 -> vw00:40548996179], (Wrapper)>[PointerTensor |

In [0]:
# define worker-based models and optimizer

vw00_model = Net().send(vw00)
vw01_model = Net().send(vw01)

vw00_optimizer = optim.SGD(vw00_model.parameters(), lr=args.lr)
vw01_optimizer = optim.SGD(vw01_model.parameters(), lr=args.lr)

models = [vw00_model, vw01_model]
params = [list(vw00_model.parameters()), list(vw01_model.parameters())]
optimizers = [vw00_optimizer, vw01_optimizer]

In [0]:
# figure out the guts of the training mechanism before putting into function.

def train_agg(epoch):
  
  for data_index in range(len(remote_dataset[0])):

    # update remote models

    for remote_index in range(len(compute_nodes)):
      print('data index: \t', data_index, '\tremote index:\t', remote_index)
      data, target = remote_dataset[remote_index][data_index]
      model = models[remote_index]
      opt = optimizers[remote_index]

      models[remote_index] = update2(data, target, model, opt)

    # encrypted aggregation
    new_params = list() # list to store encrypted model average

    for i in range(len(params[0])): # iterate through each parameter

      # for each worker
      spdz_params_th = list()
      for remote_index in range(len(compute_nodes)):

        # copy of parameter (cop): copy same parameter from each worker's model, floating type tensor
        cop = params[remote_index][i].copy()

        # copy of parameter, thousands (cop_th): copy of parameter scaled up so that precision is not lost
        # fixed precision parameter, thousands (fpp_th): fixed precision version of parameter scaled up so that precision is not lost
        fpp_th = cop * 10000

        # encrypt on the remote machine. Note: fixed_precision_param is already a pointer. 
        # calling share encrypts data to which pointer is pointing. Returns a pointer to the MPC secret shared object. Need to fetch object.
        encrypted_param_th = fpp_th.share(vw00, vw01, crypto_provider=aggr)

        # fetch the pointer to the MPC shared value
        param_th = encrypted_param_th.get()

        # save parameter so can average with same parameter from other workers.
        spdz_params_th.append(param_th)

      # average param from multiple workers, fetch back to local machine. Decrypt and decode from fixed precision back to float. 
      new_param_th = (spdz_params_th[0] + spdz_params_th[1]).get() / 2 #.float_precision() / 2 
      new_param_float_th = new_param_th.type(torch.FloatTensor) 
      new_param_float = new_param_float_th / 10000

      # save new averaged parameter
      new_params.append(new_param_float)

    print(new_params)  

    with torch.no_grad():
      for model in params:
        for param in model:
          param *= 0

      for remote_index in range(len(compute_nodes)):
        for param_index in range(len(params[remote_index])):
          params[remote_index][param_index].set_(new_params[param_index])

  for model in models:
    model.get()

In [0]:
def test_agg():
  models[0].eval()
  test_loss = 0
  for data, target in testloader:
    output = models[0](data)
    test_loss += loss_func(output, target).item()
    pred = output
  
  test_loss /= len(testloader.dataset)
  print('Test set: Average loss: {:.4f}\n'.format(test_loss))
    

In [149]:
t = time.time()

for epoch in range(args.epochs):
  print(f"Epoch {epoch + 1}")
  train(epoch)
  test()
  
total_time = time.time() - t

print('Total', round(total_time, 2), 's')

Epoch 1

Test Set: Average Loss: 0.8211

Epoch 2

Test Set: Average Loss: 0.8211

Epoch 3

Test Set: Average Loss: 0.8211

Epoch 4

Test Set: Average Loss: 0.8211

Epoch 5

Test Set: Average Loss: 0.8211

Epoch 6

Test Set: Average Loss: 0.8211

Epoch 7

Test Set: Average Loss: 0.8211

Epoch 8

Test Set: Average Loss: 0.8211

Epoch 9

Test Set: Average Loss: 0.8211

Epoch 10

Test Set: Average Loss: 0.8211

Total 0.22 s
